http://arxiv.org/abs/1409.2329

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import time

import numpy as np
import tensorflow as tf
import reader

import util
from tensorflow.python.client import device_lib

In [2]:
BASIC = "basic"
CUDNN = "cudnn"
BLOCK = "block"

In [3]:
class SmallConfig(object):
    """Small config."""
    init_scale = 0.1
    learning_rate = 1.0
    max_grad_norm = 5
    num_layers = 2
    num_steps = 20
    hidden_size = 200
    max_epoch = 4
    max_max_epoch = 8
    keep_prob = 1.0
    lr_decay = 0.5
    batch_size = 20
    vocab_size = 10000
    rnn_mode = BLOCK

In [4]:
class MediumConfig(object):
    """Medium config."""
    init_scale = 0.05
    learning_rate = 1.0
    max_grad_norm = 5
    num_layers = 2
    num_steps = 35
    hidden_size = 650
    max_epoch = 6
    max_max_epoch = 39
    keep_prob = 0.5
    lr_decay = 0.8
    batch_size = 20
    vocab_size = 10000
    rnn_mode = BLOCK

In [5]:
class LargeConfig(object):
    """Large config."""
    init_scale = 0.04
    learning_rate = 1.0
    max_grad_norm = 10
    num_layers = 2
    num_steps = 35
    hidden_size = 1500
    max_epoch = 14
    max_max_epoch = 55
    keep_prob = 0.35
    lr_decay = 1 / 1.15
    batch_size = 20
    vocab_size = 10000
    rnn_mode = BLOCK

In [6]:
class TestConfig(object):
    """Tiny config, for testing."""
    init_scale = 0.1
    learning_rate = 1.0
    max_grad_norm = 1
    num_layers = 1
    num_steps = 2
    hidden_size = 2
    max_epoch = 1
    max_max_epoch = 1
    keep_prob = 1.0
    lr_decay = 0.5
    batch_size = 20
    vocab_size = 10000
    rnn_mode = BLOCK

In [7]:
class PTBInput(object):
    """The input data"""
    
    def __init__(self, config, data, name=None):
        self.batch_size = batch_size = config.batch_size
        self.num_steps = num_steps = config.num_steps
        self.epoch_size = ((len(data)//batch_size)-1) // num_steps
        self.input_data, self.targets = reader.ptb_producer(data, batch_size,
                                                           num_steps, name=name)

In [8]:
class PTBModel(object):
    
    def __init__(self, is_training, config, input_):
        self.is_training = is_training
        self._input = input_
        self._rnn_params = None
        self._cell = None
        self.batch_size = input_.batch_size
        self.num_steps = input_.num_steps
        size = config.hidden_size
        vocab_size = config.vocab_size
        
        # 특정 작업에 특정 디바이스를 배치
        with tf.device("/cpu:0"):
            # embedding matrix를 variable로 만들 수 있다.
            embedding = tf.get_variable(name="embedding", shape=[vocab_size, size], dtype=tf.float32)
            # embedding layer
            # inputs : [batch_size, n_step, hidden_size(size)]
            inputs = tf.nn.embedding_lookup(embedding, input_.input_data)
            
        # 만약 training이고 dropout의 1이하라면
        # input단에 적용한다.
        if is_training and config.keep_prob < 1:
            inputs = tf.nn.dropout(inputs, config.keep_prob)
            
        # 여기서 RNN을 만들어서 그래프 위에 올린다
        # ouput : [batch*n_step, hidden_size]
        # states h,c : [batch, hidden_size]
        self.output, self.state = self._build_rnn_graph(inputs, config, is_training)
        
        # logits : [batch, n_step, vocab_size]를 만들기 위해 matmul(dense layer)로 보낸다.
        # 이 logits이 sequence loss로 들어간다.
        softmax_w = tf.get_variable("softmax_w", [size, vocab_size], dtype=tf.float32)
        softmax_b = tf.get_variable("softmax_b", [vocab_size], dtype=tf.float32)
        # x : 2D tensor, w : 2D tensor
        logits = tf.nn.xw_plus_b(self.output, softmax_w, softmax_b)
        logits = tf.reshape(logits, [self.batch_size,self.num_steps,vocab_size])
        
        # average_across_batch가 True이면 batch dimension을 기준으로 sum을 하고 batch_size로 나눈다.
        # average_across_timesteps가 True이면 sequence dimension을 기준으로 sum하고 
        # timestep에 대한 total label weight로 나누게 된다.
        # targets : [batch_size , n_steps]
        loss = tf.contrib.seq2seq.sequence_loss(logits,
                                                input_.targets,
                                               tf.ones([self.batch_size,self.num_steps], dtype=tf.float32),
                                               average_across_timesteps=False,
                                               average_across_batch=True)
        # 1-dimension짜리 cost로 만들어준다.
        self._cost = tf.reduce_sum(loss)
        self._final_state = self.state
        
        # test라면 모델까지만 그래프에 올린다.
        if not is_training:
            return
        
        self._lr = tf.Variable(0.0, trainable=False)
        # 그래프에 올라온 trainable한 변수를 불러온다.
        tvars = tf.trainable_variables()
        # gradient matrix의 norm을 조정한다
        grads, _ = tf.clip_by_global_norm(t_list=tf.gradients(self._cost, tvars), clip_norm=config.max_grad_norm)
                                      
        # optimizer
        optimizer = tf.train.GradientDescentOptimizer(self._lr)
        self._train_op = optimizer.apply_gradients(zip(grads, tvars), global_step=tf.train.get_or_create_global_step())
        
        # learning rate decay
        self._new_lr = tf.placeholder(tf.float32, shape=[], name="new_learning_rate")
        self._lr_update = tf.assign(self._lr, self._new_lr)
        
    def _build_rnn_graph(self, inputs, config, is_training):
    # CUDNN 모드에 따라서 다르게 동작한다. 
    # CUDNN 버전 LSTM이 존재한다.
        if config.rnn_mode == CUDNN:
            return self._build_rnn_graph_cudnn(inputs, config, is_training)
        else:
            return self._build_rnn_graph_lstm(inputs, config, is_training)
    
    def _build_rnn_graph_lstm(self, inputs, config, is_training):
        # forget gate biase를 1로 주는게 더 좋은 결과가 나오지만 원본 논문에선 그걸 고려안해서
        # 원본 논문의 hyper parameter를 쓰기 위해 1을 주지 않는다.
        def make_cell():
            cell = self._get_lstm_cell(config, is_training)
            # 만약에 training이고 keep_prob값이 1 이하라면 output connection에
            # Dropout을 걸어준다.
            if is_training and config.keep_prob < 1:
                cell = tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob=config.keep_prob)
                
            return cell
        # deep layered LSTM을 구성한다.
        cell = tf.contrib.rnn.MultiRNNCell([make_cell() for _ in range(config.num_layers)], state_is_tuple=True)
        
        # [batch_size x hidden_size]
        self._initial_state = cell.zero_state(config.batch_size, tf.float32)
        state = self._initial_state
        
        
        self.outputs = []
        with tf.variable_scope("RNN"):
            for time_step in range(self.num_steps):
                # time_step이 1인 이후부터는 cell의 wieght를 다시 이용
                # inputs : [batch_size, n_step, hidden_size(size)]
                if time_step > 0: tf.get_variable_scope().reuse_variables()
                (cell_output, state) = cell(inputs[:, time_step, :], state)
                self.outputs.append(cell_output)
                
            # outputs : [ n_step, batch, hidden]
            # tf.concat(self.outputs, 1) [n_steps, hidden*batch]
            # self.concat = tf.concat(self.outputs, 1)
            output = tf.reshape(tf.concat(self.outputs, 1), [-1, config.hidden_size])
            # output [batch_size*n_step, hidden_size]
            return output, state
    
    # CUDNN모드의 LSTM이 아니면 basic버전이나 block버전의 cell을 만든다. 
    def _get_lstm_cell(self, config, is_training):
        if config.rnn_mode == BASIC:
            return tf.nn.rnn_cell.BasicLSTMCell(config.hidden_size, forget_bias=0.0, reuse=not is_training)
        if config.rnn_mode == BLOCK:
            return tf.contrib.rnn.LSTMBlockCell(config.hidden_size, forget_bias=0.0)
        raise ValueError("rnn_mode %s not supported" % config.rnn_mode)
        
    def assign_lr(self, session, lr_value):
        session.run(self._lr_update, feed_dict={self._new_lr: lr_value})
            
     
    # getters
    @property
    def input(self):
        return self._input

    @property
    def initial_state(self):
        return self._initial_state

    @property
    def cost(self):
        return self._cost

    @property
    def final_state(self):
        return self._final_state

    @property
    def lr(self):
        return self._lr

    @property
    def train_op(self):
        return self._train_op

    @property
    def initial_state_name(self):
        return self._initial_state_name

    @property
    def final_state_name(self):
        return self._final_state_name
    
    # Parallel Processing을 하지 않으면 필요없다.
    def export_ops(self, name):
        """Exports ops to collections."""
        # name으로 오는 값이 "Train", "Valid", "Test"이다.
        # 각각 Train, Valid, Test마다 ops의 collection을 따로 주려고 한다.
        # self._name/cost : self._cost의 dictinary를 만든다.
        self._name = name
        ops = {util.with_prefix(self._name, "cost"): self._cost}
        # training mode라면 learning rate관련 ops도 dict에 추가한다.
        if self._is_training:
            ops.update(lr=self._lr, new_lr=self._new_lr, lr_update=self._lr_update)
            # cudnn 모드용 lstm cell을 만들어서 이에 관련된 ops가 있다면
            # 이것도 추가한다.
            if self._rnn_params:
                ops.update(rnn_params=self._rnn_params)
        
        for name, op in ops.items():
            # 모아놓은 op에 name이라는 collection으로 등록한다.
            tf.add_to_collection(name, op)
                
        #lstm의 initial state와 final state도 collection을 등록한다.
        # self._name/initial
        self._initial_state_name = util.with_prefix(self._name, "initial")
        # self._name/final
        self._final_state_name = util.with_prefix(self._name, "final")
        # lstm의 initial c, h sate 
        util.export_state_tuples(self._initial_state, self._initial_state_name)
        # lstm의 final c, h sate
        util.export_state_tuples(self._final_state, self._final_state_name)

```python
m._input.targets
<tf.Tensor 'Train/TrainInput/StridedSlice_1:0' shape=(20, 20) dtype=int32>


m.initial_state

(LSTMStateTuple(c=<tf.Tensor 'Train/Model/MultiRNNCellZeroState/LSTMBlockCellZeroState/zeros:0' shape=(20, 200) dtype=float32>, h=<tf.Tensor 'Train/Model/MultiRNNCellZeroState/LSTMBlockCellZeroState/zeros_1:0' shape=(20, 200) dtype=float32>), LSTMStateTuple(c=<tf.Tensor 'Train/Model/MultiRNNCellZeroState/LSTMBlockCellZeroState_1/zeros:0' shape=(20, 200) dtype=float32>, h=<tf.Tensor 'Train/Model/MultiRNNCellZeroState/LSTMBlockCellZeroState_1/zeros_1:0' shape=(20, 200) dtype=float32>))


m._cost
Tensor("Train/Model/Sum:0", shape=(), dtype=float32)


m.output
<tf.Tensor 'Train/Model/RNN/Reshape:0' shape=(400, 200) dtype=float32>

m.state
(LSTMStateTuple(c=<tf.Tensor 'Train/Model/RNN/RNN/multi_rnn_cell/cell_0/lstm_cell/LSTMBlockCell_19:1' shape=(20, 200) dtype=float32>, h=<tf.Tensor 'Train/Model/RNN/RNN/multi_rnn_cell/cell_0/lstm_cell/LSTMBlockCell_19:6' shape=(20, 200) dtype=float32>),
 LSTMStateTuple(c=<tf.Tensor 'Train/Model/RNN/RNN/multi_rnn_cell/cell_1/lstm_cell/LSTMBlockCell_19:1' shape=(20, 200) dtype=float32>, h=<tf.Tensor 'Train/Model/RNN/RNN/multi_rnn_cell/cell_1/lstm_cell/LSTMBlockCell_19:6' shape=(20, 200) dtype=float32>))


m.concat
<tf.Tensor 'Train/Model/RNN/concat:0' shape=(20, 4000) dtype=float32>
```

In [10]:
def run_epoch(session, model, eval_op=None, verbose=False):
    """epoch하나를 실행시켜서 model을 돌리는 함수
    출력값으로 perplexity를 출력한다."""
    start_time = time.time()
    costs = 0.0
    iters = 0
    # initial state를 0으로 초기화시킨다.
    state = session.run(model.initial_state)
    
    fetches = {
      "cost": model.cost,
      "final_state": model.final_state,}
    
    # Train 모델만 m.train_op를 eval_op로 넘겨준다.
    # Train일 경우에 eval_op를 fetch dict에 추가한다
    if eval_op is not None:
        fetches["eval_op"] = eval_op
        
    for step in range(model.input.epoch_size):
        feed_dict = {}
        # lstm의 initial state
        for i, (c,h) in enumerate(model.initial_state):
            feed_dict[c] = state[i].c
            feed_dict[h] = state[i].h
        
        # model.cost, model_final_state, m.train_op를 돌린다.
        # fetches가 dict이기 때문에 vals은 dict로 나온다.
        vals = session.run(fetches, feed_dict)
        cost = vals["cost"]
        state = vals["final_state"]
        
        costs += cost
        iters += model.input.num_steps
        
        if verbose and step % (model.input.epoch_size // 10) == 10:
            print("%.3f perplexity: %.3f speed: %.0f wps" %
                  (step * 1.0 / model.input.epoch_size, np.exp(costs / iters),
                   iters * model.input.batch_size/
                   (time.time() - start_time)))
            
    return np.exp(costs / iters)

In [11]:
raw_data = reader.ptb_raw_data('D:\PythonLab\CS20\RNN')
train_data, valid_data, test_data, _ = raw_data
print(np.shape(train_data))

(929589,)


In [12]:
config = SmallConfig()

In [13]:
eval_config = SmallConfig()
eval_config.batch_size = 1
eval_config.num_steps = 1

In [14]:
initializer = tf.random_uniform_initializer(-config.init_scale,config.init_scale)
    
# Train용 모델을 만든다.
with tf.name_scope("Train"):
    # train_input.input_data : [batch_size, n_step] = [20,20]
    # train_input.targets : [batch_size, n_step] = [20,20]
    # train_input.epoch_size : 2323
    train_input = PTBInput(config=config, data=train_data, name="TrainInput")
    # variable_scope로 묶어서 initializer를 한꺼번에 적용하자
    with tf.variable_scope("Model", reuse=None, initializer=initializer):
        m = PTBModel(is_training=True, config=config, input_=train_input)
    tf.summary.scalar("Training_Loss", m.cost)
    tf.summary.scalar("Learning_Rate", m.lr)
    

Instructions for updating:
Use the retry module or similar alternatives.


In [ ]:
# Validation용 모델을 만든다
with tf.name_scope('Valid'):
    # valid_input.input_data : [batch_size, n_step] = [20,20]
    # valid_input.targets : [batch_size, n_step] = [20,20]
    # valid_input.epoch_size : 184
    valid_input = PTBInput(config=config, data=valid_data, name="ValidInput")
    # 앞에서 만든 Train model을 재활용한다.
    with tf.variable_scope("Model", reuse=True, initializer=initializer):
        mvalid = PTBModel(is_training=False, config=config, input_=valid_input)
    tf.summary.scalar("Validation_Loss", mvalid.cost)

In [ ]:
# Test용 모델을 만든다
with tf.name_scope("Test"):
    test_input = PTBInput(config=eval_config, data=test_data, name="TestInput")
    with tf.variable_scope("Model", reuse=True, initializer=initializer):
        mtest = PTBModel(is_training=False, config=eval_config,input_=test_input)

In [ ]:
# models = {"Train": m, "Valid": mvalid, "Test": mtest}
# for name, model in models.items():
#     # learning_rate ops, final, initaial state, cost를 collection에 등록한다.
#     model.export_ops(name)
# metagraph = tf.train.export_meta_graph()

In [ ]:
save_path = 'D:\PythonLab\CS20\RNN'

In [18]:
# High-level Monitored Session
sv = tf.train.Supervisor(logdir=save_path)
#sv = tf.train.MonitoredTrainingSession(checkpoint_dir='D:\PythonLab\CS20\RNN')
config_proto = tf.ConfigProto(gpu_options=tf.GPUOptions(allow_growth=True))
with sv.managed_session(config=config_proto) as session:
    for i in range(config.max_max_epoch):
        # 왜 max_epoch과 max_max_epoch이 다를까?
        # 논문에서 제시된 epoch당 줄이는 factor값을 맞추기 위해서
        lr_decay = config.lr_decay ** max(i+1-config.max_epoch, 0.0)
        # Tranin Model의 learning rate decay하는 operation 실행
        m.assign_lr(session, config.learning_rate * lr_decay)
        
        print("Epoch: %d Learning rate: %.3f" % (i + 1, session.run(m.lr)))
        # Train
        train_perplexity = run_epoch(session, m, eval_op=m.train_op,verbose=True)
        print("Epoch: %d Train Perplexity: %.3f" % (i + 1, train_perplexity))
        
        # Valid
        valid_perplexity = run_epoch(session, mvalid)
        print("Epoch: %d Valid Perplexity: %.3f" % (i + 1, valid_perplexity))
        
    # Test
    test_perplexity = run_epoch(session, mtest)
    print("Test Perplexity: %.3f" % test_perplexity)
    # Model 저장
    print("Saving model to %s." % save_path)
    sv.saver.save(session, save_path, global_step=sv.global_step)

Instructions for updating:
Please switch to tf.train.MonitoredTrainingSession
INFO:tensorflow:Restoring parameters from D:\PythonLab\CS20\RNN\model.ckpt-0
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Starting standard services.
INFO:tensorflow:Saving checkpoint to path D:\PythonLab\CS20\RNN\model.ckpt
INFO:tensorflow:Starting queue runners.
INFO:tensorflow:Model/global_step/sec: 0
Epoch: 1 Learning rate: 1.000
INFO:tensorflow:Recording summary at step 0.
0.004 perplexity: 5386.678 speed: 6207 wps
0.104 perplexity: 836.998 speed: 10822 wps
0.204 perplexity: 623.433 speed: 11250 wps
0.304 perplexity: 504.161 speed: 11380 wps
0.404 perplexity: 435.992 speed: 11526 wps
0.504 perplexity: 390.409 speed: 11469 wps
0.604 perplexity: 351.864 speed: 11509 wps
0.703 perplexity: 325.033 speed: 11474 wps
0.803 perplexity: 303.831 speed: 11509 wps
0.903 perplexity: 284.428 speed: 11517 wps
Epoch: 1 Train Perplexity: 270.040
Epoch: 1 Valid Perplex

INFO:tensorflow:Recording summary at step 30026.
INFO:tensorflow:Model/global_step/sec: 27.4074
Epoch: 13 Train Perplexity: 40.711
Epoch: 13 Valid Perplexity: 120.142
INFO:tensorflow:Recording summary at step 30199.
INFO:tensorflow:Saving checkpoint to path D:\PythonLab\CS20\RNN\model.ckpt
Test Perplexity: 114.624
Saving model to D:\PythonLab\CS20\RNN.
